In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from tqdm import tqdm
from PIL import ImageFile

ImageFile.LOAD_TRUNCATED_IMAGES = True

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ====== 1. Dataset Paths ======
data_dir = "/workspace//dataset/Bone_Fracture_Binary_Classification/Bone_Fracture_Binary_Classification/"
train_dir = os.path.join(data_dir, "train")
val_dir = os.path.join(data_dir, "val")
test_dir = os.path.join(data_dir, "test")

# ====== 2. Transforms & Loaders ======
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

train_loader = DataLoader(
    datasets.ImageFolder(train_dir, transform=transform),
    batch_size=32, shuffle=True
)
val_loader = DataLoader(
    datasets.ImageFolder(val_dir, transform=transform),
    batch_size=32, shuffle=False
)
test_loader = DataLoader(
    datasets.ImageFolder(test_dir, transform=transform),
    batch_size=32, shuffle=False
)

# ====== 3. CNN Model ======
class CNNClassifier(nn.Module):
    def __init__(self):
        super(CNNClassifier, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(32 * 56 * 56, 128),
            nn.ReLU(),
            nn.Linear(128, 2)
        )

    def forward(self, x):
        return self.model(x)

model = CNNClassifier().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# ====== 4. Training ======
epochs = 5
for epoch in range(epochs):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    for imgs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        preds = torch.argmax(outputs, dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    print(f"Train Loss: {total_loss:.4f}, Accuracy: {100 * correct / total:.2f}%")

# ====== 5. Evaluation ======
def evaluate(loader, name):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for imgs, labels in loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            preds = torch.argmax(outputs, dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    print(f"{name} Accuracy: {100 * correct / total:.2f}%")

evaluate(train_loader, "Train")
evaluate(val_loader, "Validation")
evaluate(test_loader, "Test")

# ====== 6. Save Model ======
torch.save(model.state_dict(), "fracture_classifier.pth")
print("✅ Model saved as 'fracture_classifier.pth'")

FileNotFoundError: [WinError 3] The system cannot find the path specified: '/workspace//dataset/Bone_Fracture_Binary_Classification/Bone_Fracture_Binary_Classification/train'